In [1]:
import ktrain
from ktrain import text
import pandas as pd
import tensorflow as tf

In [2]:
from distutils.version import LooseVersion
assert LooseVersion(tf.__version__) >= LooseVersion('1.0')
print('TensorFlow Version: {}'.format(tf.__version__))

if not tf.test.gpu_device_name():
    print('NO GPU FOUND')
else:
    print('GPU: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 2.1.0
GPU: /device:GPU:0


In [3]:
df_train = pd.read_csv('df_train.csv')
df_train = df_train[df_train.text.notna()]
df_test = pd.read_csv('df_test.csv')
df_test = df_test[df_test.text.notna()]

In [7]:
df_merged = pd.concat([df_train, df_test])

In [12]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(df_merged, 
                                                                   'text', # name of column containing review text
                                                                   label_columns=['rating'],
                                                                   maxlen=50, 
                                                                   max_features=50,
                                                                   preprocess_mode='bert',
                                                                   val_pct=0.2,
                                                                   ngram_range=3)

model = text.text_classifier('bert', (x_train, y_train) , preproc=preproc)
learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=32)

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


Is Multi-Label? False
maxlen is 50
done.


In [13]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 2919980 samples, validate on 729995 samples
2919980/2919980 [==============================] - 55243s 19ms/sample - loss: 0.6799 - accuracy: 0.7086 - val_loss: 0.6197 - val_accuracy: 0.7368


In [14]:
predictor = ktrain.get_predictor(learner.model, preproc)
data=["There is no reason to drop $60 on this dumpster fire. Just go to YouTube and skip all the BS.",
    "Great game so far. Was a huge fan of the first one. Anyone who trashes the game because of LBGT references is a horrible person leaving low reviews because of hate. If you liked the first game, you'll like this one just as much, if not more."]

In [15]:
predictor.predict(data)

D:\steffanini\anaconda\envs\gputest\lib\site-packages\ktrain\text\preprocessor.py:228: UserWarning: List or array of two texts supplied, so task being treated as text classification. If this is a sentence pair classification task, please cast to tuple.
  warnings.warn('List or array of two texts supplied, so task being treated as text classification. ' +\


['bad', 'good']

In [16]:
predictor.save('sentiment_analysis')

In [2]:
predictor = ktrain.load_predictor('sentiment_analysis')
data=["I compared these to my other reverse weave gray champion joggers I have and I think they’re not legit! I bought a large gray one in store and they fit nice and cinched at my waist and fall on my belly button but these that I ordered through here do not fit the same!!!!! You would think if it’s the same product it would fit the same size but just a different color. I am so upset because I already took the tag off and I cannot return them, I thought that after a wash it would fit better but no I was wrong. I am so upset with this purchase and will not be buying again. I attached two photos. The gray pants i bought from a verified retailer of champion it was made in Honduras, and the black champion joggers I bought from amazon had tag saying made in Pakistan, also each tag has a different material feel to it. IF IT IS THE SAME JOGGER WHY MADE IN DIFF PLACES? This is why I believe the black joggers I bought from this retailer is not actually champion. I wish I could get my money back.",
     "Love love loved these pants... until I wash and dried them. They shrank so much!! Would not recommend putting them in the dryer!",
     "Likes these but Ran a bit small; needed next size but they rose in price so much I couldn’t re-order. Had to return but sturdy nice material. Also Just a bit short for my 5’9” and 5’10” girls.",
     "These are my FAVORITE sweat pants, they’re comfy enough to lay around in but also look fitted enough to run errands and leave the house in. Good, sturdy material doesn’t stretch and look saggy. I’m 5’7 145 and I ordered a medium and they’re perfect. These will shrink a bit if you dry them."]
predictor.predict(data)

['good', 'bad', 'neutral', 'good']